In [8]:
import os
import sys
import time
import requests

from datetime import datetime
from pyspark.sql import SparkSession

# Pemrosesan Data Menggunakan Apache Spark

Percobaan kali ini secara proses hampir sama seperti yang dilakukan pada proses etl sebelumnya, namun satu yang berbeda adalah penggunaan *Apache Spark* untuk pemrosesan data. Yang mana sebelumnya hanya menggunakan dataframe dari pandas. Kekurangan dari penggunaan pandas adalah pada *Eager Execution* dimana ketika butuh proses transformasi yang berulang akan cukup merepotkan. Oleh karena itulah disini spark digunakan, dimana kita bisa menyusun tahapan-tahapan transformasi terlebih dahulu tanpa langsung melakukan transformasi secara eksplisit.

Ada yang berbeda juga kali ini dimana data sebelumnya ada di kota *San Fransisco*, untuk kali ini kami melakukan beberapa penyesuaian yaitu dengan merubah tempat menjadi *Kota Malang*. Adapun untuk sumber data yang kami gunakan masih tetap berasal dari API Open-meteo (https://open-meteo.com/en/docs/historical-weather-api)

In [9]:
# Tambahkan 2 baris ini agar spark mengenali env python
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [10]:
# Buat spark session terlebih dahulu
spark = SparkSession.builder \
    .appName("rdv-project") \
    .master("local[*]") \
    .getOrCreate()

# Extract Data

Ekstraksi data yang dilakukan juga akan berbeda dari tugas sebelumnya. Pada tugas sebelumnya hanya menggunakan satu fitur saja yaitu temperature_2m. Namun pada kali ini karena spark memanfaatkan memori untuk pemrosesan data, kami akan mengambil lebih banyak fitur.

List fitur yang akan dipakai : 
- `temperature_2m`,
- `relative_humidity_2m`,
- `precipitation`,
- `weather_code`,
- `wind_speed_10m`,
- `wind_speed_100m`,
- `wind_direction_10m`,
- `wind_direction_100m`,
- `wind_gusts_10m`,
- `rain`,
- `apparent_temperature`,
- `surface_pressure`,
- `cloud_cover`,
- `cloud_cover_low`,
- `cloud_cover_mid`,
- `cloud_cover_high`,
- `soil_temperature_0_to_7cm`,
- `soil_temperature_7_to_28cm`,
- `soil_temperature_28_to_100cm`,
- `soil_temperature_100_to_255cm`,
- `soil_moisture_0_to_7cm`,
- `soil_moisture_7_to_28cm`,
- `soil_moisture_28_to_100cm`,
- `soil_moisture_100_to_255cm`,
- `et0_fao_evapotranspiration`,
- `vapour_pressure_deficit`,
- `pressure_msl`,
- `dew_point_2m`

Bisa dilihat bahwa fitur yang diambil bisa dibilang cukup banyak. Jika hanya untuk dashboard saja yang tujuannya untuk monitoring cuaca saat ini, fitur-fitur seperti informasi keadaan tanah dan lain-lain yang terlalu spesifik mungkin tidak diperlukan. Akan tetapi kebutuhan akan berbeda ketika ada pertimbangan untuk melakukan **prediksi** kode cuaca (*weather code*) dari semua fitur yang ada.

In [11]:
"""
Malang sendiri sebenarnya tidak memiliki stasiun pengukuran dari BMKG
Oleh karena itu latitude dan longitude di bawah ini menunjukkan 
koordinat dari kota malang seluruhnya
"""
# --------------------------- Parameter API -------------------------

# Latitude & Longitude utk Malang
latitude = -7.96611694302791
longitude = 112.62816916904798

# Tanggal
start_date = "2024-01-01"
today = datetime.today()
today_str = today.strftime("%Y-%m-%d")
end_date = today_str

# List Fitur
feature_list = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "weather_code",
    "wind_speed_10m",
    "wind_speed_100m",
    "wind_direction_10m",
    "wind_direction_100m",
    "wind_gusts_10m",
    "rain",
    "apparent_temperature",
    "surface_pressure",
    "cloud_cover",
    "cloud_cover_low",
    "cloud_cover_mid",
    "cloud_cover_high",
    "soil_temperature_0_to_7cm",
    "soil_temperature_7_to_28cm",
    "soil_temperature_28_to_100cm",
    "soil_temperature_100_to_255cm",
    "soil_moisture_0_to_7cm",
    "soil_moisture_7_to_28cm",
    "soil_moisture_28_to_100cm",
    "soil_moisture_100_to_255cm",
    "et0_fao_evapotranspiration",
    "vapour_pressure_deficit",
    "pressure_msl",
    "dew_point_2m"
]
features = ",".join(feature_list)

In [14]:
final_psdf = None

for year in range(2017, 2025+1):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    if year==2025:
        end_date = f"{year}-05-11"

    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly={features}&timezone=auto"

    success = False
    attempts = 0

    while not success and attempts < 3:
        try:
            print(f"Getting data for {year}")
            res = requests.get(url)
            if res.status_code == 200:
                data = res.json()
                data = data['hourly']
                psdf = spark.read.json(spark.sparkContext.parallelize(data))

                if final_psdf is None:
                    final_psdf = psdf
                else :
                    final_psdf = final_psdf.unionByName(psdf)
                
                success = True
                time.sleep(60)
            elif res.status_code == 429:
                print(f"Limit rate has been reached, retrying...")
                time.sleep(60)
                attempts += 1
                print(f"#### Attempts : {attempts+1}/3 ####")
            elif res.status_code == 443:
                print(f"Error status code {res.status_code}, retrying...")
                time.sleep(60)
                attempts += 1
                print(f"#### Attempts : {attempts+1}/3 ####")
            else :
                print(f"Failed to get data on {year}. status : {res.status_code}")
                break
        except Exception as e:
            print(f"Error while get data on {year}, retrying...")
            time.sleep(60)
            attempts += 1
            print(f"#### Attempts : {attempts+1}/3 ####")

Getting data for 2017
Getting data for 2018
Getting data for 2019
Getting data for 2020
Getting data for 2021
Getting data for 2022
Getting data for 2023
Getting data for 2024
Getting data for 2025


# Transform Data

## Structuring

Tahap structuring sudah dilakukan bersamaan dengan ekstrak data dari API open-meteo. Adapun pada section structuring disini akan dilakukan pengecekan kolom pada data yang telah diambil sebelumnya.

In [15]:
final_psdf.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|                time|
|      temperature_2m|
|relative_humidity_2m|
|       precipitation|
|        weather_code|
|      wind_speed_10m|
|     wind_speed_100m|
|  wind_direction_10m|
| wind_direction_100m|
|      wind_gusts_10m|
|                rain|
|apparent_temperature|
|    surface_pressure|
|         cloud_cover|
|     cloud_cover_low|
|     cloud_cover_mid|
|    cloud_cover_high|
|soil_temperature_...|
|soil_temperature_...|
|soil_temperature_...|
+--------------------+
only showing top 20 rows



ternyata masih ada kesalahan dalam strukturisasi dataframe, dimana data masih tersusun selayaknya list dengan satu kolom saja.

## Cleaning

## Validating

# Load Data